## Preparação dos dados EOG

Neste notebook está incluído os seguintes passos:
- Realização do carregamento do *dataset*;
- Aplicação dos filtros temporais (~ 5-50 Hz) - deixado como **tarefa**;
- Segmentação de dados nos domínios do tempo e da frequência em janelas deslizantes com sobreposição.

In [3]:
# Libraries
import numpy as np
from scipy import signal
from scipy.signal import stft

In [4]:
# Load data
file_data = np.load("dataset/Gabi.npy")
file_data.shape

(28, 1000, 4)

In [5]:
# Correção do número de eletrodos
# os dois primeiros eletrodos são funcionais

data = file_data[:, :, :2]
data = data.transpose(0, 2, 1)
data.shape

(28, 2, 1000)

*Tarefa 1*: Realizem a aplicação dos filtros para toda a base.

In [6]:
# funcões para aplicação dos filtros temporais

def butter_bandpass(data, lowcut, highcut, fs=512, order=4):
    nyq = fs * 0.5
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='bandpass')
    return signal.filtfilt(b, a, data)

def butter_notch(data, cutoff, var=1, fs=512, order=4):
    nyq = fs * 0.5
    low = (cutoff - var) / nyq
    high = (cutoff + var) / nyq
    b, a = signal.iirfilter(order, [low, high], btype='bandstop', ftype="butter")
    return signal.filtfilt(b, a, data)

In [7]:
data = butter_notch(data, 60)
data = butter_bandpass(data, 5, 50)

data.shape

(28, 2, 1000)

#### Segmentação dos dados

As características normalmente são extraídas sobre pequenos segmentos de tamanho fixo dos dados, não no dado como um todo. Estes pequenos segmentos são chamados de janelas. A técnica de separar os dados em janelas, recebe o nome de janela deslizante (*sliding window*) e é uma forma de segmentação de dados. Uma boa prática, é definir um passo para essa janela de forma que haja uma sopreposição de dados, para que informações da lacuna de uma janela e outra não sejam perdidas. Na imagem a seguir podemos observar um sinal EOG.

Algumas características de EMG trabalham com o dado no domínio da frequência. Quando tais características são aplicadas, é necessário tranformar o dado para o domínio da frequência, utilizando o método de transformação de domínio `STFT`, do inglês *Short-time Fourier transform* (Transformada de Fourier de curto termo). 

Os trechos de código a seguir dividem o dadom em segmentos, tanto no domínio do tempo quanto no domínio da frequência.

In [8]:
step = 29
segment = 64
print('Shape inicial:', data.shape)

n_win = int((data.shape[-1] - segment) / step) + 1
ids = np.arange(n_win) * step

# Janelas do dado no dominio do tempo
chunks_time = np.array([data[:,:,k:(k + segment)] for k in ids]).transpose(1, 2, 0, 3)

# # Janelas do dado no domínio da frequência
_, _, chunks_freq = stft(data, fs=200, nperseg=64, noverlap=32)
chunks_freq = np.swapaxes(chunks_freq, 2, 3)

print('Formato (shape) dos dados depois da divisão de janelas')
print(f'Dominio do tempo: {chunks_time.shape} - (classes+participantes, canais, janelas, linhas)')
print(f'Dominio da frequência:  {chunks_freq.shape} - (classes+ensaios, canais, janelas, linhas)')

Shape inicial: (28, 2, 1000)
Formato (shape) dos dados depois da divisão de janelas
Dominio do tempo: (28, 2, 33, 64) - (classes+participantes, canais, janelas, linhas)
Dominio da frequência:  (28, 2, 33, 33) - (classes+ensaios, canais, janelas, linhas)


*Tarefa 2*: separar os participantes e armazenar os dados em disco para a execução das próximas tarefas.

In [9]:
# Salvamento simples dos dados como está, para uso no notebook "validation.ipynb" (somente professor).
np.save("dataset/data_time.npy", chunks_time)
np.save("dataset/data_freq.npy", chunks_freq)